In [ ]:
import pandas as pd
import re
import nltk

In [ ]:
data =pd.read_csv('/kaggle/input/stockheadings/Data.csv', encoding="ISO-8859-1")

In [ ]:
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

In [ ]:
nltk.download('wordnet')

In [ ]:
data.head(2)

In [ ]:
train = data[data['Date'] < '20150101']
test = data[data['Date'] > '20141231']

In [ ]:
x_train=train.iloc[:,2:27]
y_train=train.iloc[:,1:2]

x_test=test.iloc[:, 2:27]
y_test=test.iloc[:,1:2]

In [ ]:
len(x_train.columns)==len(x_test.columns)

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
lem=WordNetLemmatizer()

In [ ]:
def process_data(x):
    x['headings']=x.apply(lambda x: ' '.join(map(str,x)), axis=1)
    x=x['headings']
    for i in range(len(x)):
        words=re.sub('[^a-zA-Z]', ' ', x.iloc[i])
        words=nltk.word_tokenize(words)
        words=[lem.lemmatize(word.lower()) for word in words if word not in stopwords.words('english')]
        x.iloc[i]=' '.join(words)
    return x

In [ ]:
x_train=process_data(x_train)

In [ ]:
x_train.head(10)

In [ ]:
x_test=process_data(x_test)

In [ ]:
x_train[0]

In [ ]:
len(x_test)==len(y_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

cv=CountVectorizer(max_features=3000, ngram_range=(1,3))

In [ ]:
x_train_vec=cv.fit_transform(x_train)

In [ ]:
x_test_vec=cv.transform(x_test)

In [ ]:
x_train_vec.shape

In [ ]:
from xgboost import XGBClassifier

model=XGBClassifier(n_estimators=400)

In [ ]:
model.fit(x_train_vec, y_train)

In [ ]:
x_test.info()

In [ ]:
y_pred=model.predict(x_test_vec)

In [ ]:
len(y_test)==len(y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy: ", accuracy_score(y_test, y_pred))